# 🎯 Problem Statement: Intelligent Review Response Generation using LangGraph Conditional Workflow

## 🧩 Objective

Build a LangGraph workflow that **reads a customer review**, performs **sentiment analysis**, and conditionally creates a personalized response based on whether the sentiment is positive or negative. If the sentiment is negative, the workflow should **deeply analyze the review** before drafting the response.

---

## 📌 Goals

1. Accept a **review text** as input.
2. Use an **LLM node** to:
   - Determine sentiment: `positive` or `negative`
   - Output structured JSON:  
     ```json
     { "sentiment": "positive" }
     ```
3. Based on sentiment, use **conditional routing**:
   - If **positive** →  
     → Go to **Response Generator Node** to generate a friendly reply  
     → **End Flow**
   - If **negative** →  
     → Go to **Review Diagnostic Node**

4. The **Review Diagnostic Node** (LLM):
   - Extract 3 key insights from the review:
     - `issue_type` (e.g., "billing", "product defect")
     - `tone` (e.g., "angry", "disappointed")
     - `urgency` (e.g., "low", "medium", "high")
   - Output JSON:
     ```json
     {
       "issue_type": "delivery delay",
       "tone": "frustrated",
       "urgency": "high"
     }
     ```

5. Use these extracted insights to:
   - Trigger a final **LLM Response Generator Node** that crafts an **empathetic and actionable reply**.

6. End the workflow after sending the response.

---

## 🔁 Workflow Overview (Node Flow)

1. **Input Node**  
   - Receives the customer review text

2. **Sentiment Analysis Node** (LLM)  
   - Outputs sentiment in structured JSON

3. **Conditional Router Node**  
   - If `sentiment == "positive"` → `PositiveReplyNode`  
   - If `sentiment == "negative"` → `DiagnosticNode`

4. **DiagnosticNode** (LLM)  
   - Extracts `issue_type`, `tone`, and `urgency`

5. **NegativeReplyNode** (LLM)  
   - Crafts customized reply based on diagnostic output

6. **Output Node**  
   - Returns the response

---

## ✅ Success Criteria

- Accurate sentiment classification using LLM.
- Structured JSON outputs at each step.
- Intelligent routing via LangGraph's conditional edges.
- Empathetic, situation-specific final responses.
- Fully automated and adaptive workflow with minimal hardcoding.

---

In [25]:
from langgraph.graph import StateGraph, END, START
from langchain_openai import ChatOpenAI
from typing import TypedDict, Literal
from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv()

True

In [26]:
model = ChatOpenAI(model = "gpt-4o-mini")

In [27]:
class SentimentSchema(BaseModel):
    sentiment : Literal["Positive", "Negative"] = Field(description="Sentiment of the review")


In [28]:
class DiagnosisSchema(BaseModel):
    issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description='The category of issue mentioned in the review')
    tone: Literal["angry", "frustrated", "disappointed", "calm"] = Field(description='The emotional tone expressed by the user')
    urgency: Literal["low", "medium", "high"] = Field(description='How urgent or critical the issue appears to be')

In [29]:
structured_model = model.with_structured_output(SentimentSchema)
structured_model2 = model.with_structured_output(DiagnosisSchema)

In [30]:
# prompt = " What is the sentiment of the following review: the software is very good"

# structured_model.invoke(prompt).sentiment

In [31]:
class ReviewState(TypedDict):
    review : str
    sentiment : Literal['Positive', "Negative"]
    diagnosis : dict
    response : str

In [32]:
def find_sentiment (state: ReviewState):
    prompt = f'for the following the review find out the sentiment \n {state['review']}'
    sentiment = structured_model.invoke(prompt).sentiment

    return {'sentiment': sentiment}


In [45]:
def check_sentiment(state:ReviewState) -> Literal["positive_response","run_diagnosis"]:
    if state['sentiment'] =="Positive":
        return "positive_response"
    else:
        return "run_diagnosis"

In [46]:
def positive_response(state:ReviewState):
    prompt = f"""Write a warm thank-you message in response to this review:
    \n\n\"{state['review']}\"\n
Also, kindly ask the user to leave feedback on our website."""
    
    response = model.invoke(prompt).content

    return {'response': response}


def run_diagnosis(state: ReviewState):

    prompt = f"""Diagnose this negative review:\n\n{state['review']}\n"
    "Return issue_type, tone, and urgency.
"""
    response = structured_model2.invoke(prompt)

    return {'diagnosis': response.model_dump()}

def negative_response(state:ReviewState):
     diagnosis = state['diagnosis']
     prompt = f"""You are a support assistant.
The user had a '{diagnosis['issue_type']}' issue, sounded '{diagnosis['tone']}', and marked urgency as '{diagnosis['urgency']}'.
Write an empathetic, helpful resolution message.
"""
     response = model.invoke(prompt).content

     return {'response': response}


In [47]:
graph = StateGraph(ReviewState)

graph.add_node("find_sentiment", find_sentiment)
graph.add_node("positive_response", positive_response)
graph.add_node("run_diagnosis", run_diagnosis)
graph.add_node("negative_response", negative_response)

graph.add_conditional_edges('find_sentiment',check_sentiment)

graph.add_edge('positive_response', END)

graph.add_edge('run_diagnosis', 'negative_response')
graph.add_edge('negative_response', END)

graph.add_edge(START, "find_sentiment")


In [ ]:
graph.compile()